# TeamMoa 부하 테스트 결과 시각화

이 노트북은 부하 테스트 결과를 차트로 생성합니다.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 기본 설정 (한글 폰트 불필요 - 영어만 사용)
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# summary.txt에서 추출한 정확한 데이터 (로그인 제외)
data = {
    'Test': ['Test 1\n(20 users)', 'Test 2\n(50 users)', 'Test 3\n(100 users)', 'Test 4\n(150 users)'],
    'Users': [20, 50, 100, 150],
    'Total Requests': [3321, 8321, 16373, 28888],  # 로그인 제외
    'Failures': [0, 0, 0, 181],  # 로그인 실패 제외
    'RPS': [5.57, 13.95, 27.46, 40.34],
    'Avg Response (ms)': [43, 42, 52, 70],  # 로그인 제외 평균
    '95%ile (ms)': [65, 68, 69, 76],
    'Error Rate (%)': [0.00, 0.00, 0.00, 0.63]  # 181/28888 = 0.63%
}

df = pd.DataFrame(data)
print("📊 TeamMoa Load Test Data Summary (Login Excluded)\n")
print(df.to_string(index=False))
print(f"\n✅ Total Requests Across All Tests: {df['Total Requests'].sum():,}")
print(f"✅ Total Failures: {df['Failures'].sum()}")
print(f"✅ Overall Error Rate: {df['Failures'].sum() / df['Total Requests'].sum() * 100:.2f}%")
print(f"\n📝 Note: Login API excluded (initial authentication only)")

In [ ]:
# 그래프 생성
fig, ax1 = plt.subplots(figsize=(14, 7))

# RPS (막대)
x = range(len(df))
bars = ax1.bar(x, df['RPS'], alpha=0.7, color='steelblue', label='RPS', width=0.6)
ax1.set_xlabel('Concurrent Users', fontsize=13, fontweight='bold')
ax1.set_ylabel('RPS (Requests Per Second)', fontsize=13, color='steelblue', fontweight='bold')
ax1.tick_params(axis='y', labelcolor='steelblue', labelsize=11)
ax1.set_xticks(x)
ax1.set_xticklabels(df['Test'], fontsize=11)
ax1.grid(axis='y', alpha=0.3, linestyle='--', linewidth=0.8)
ax1.set_ylim(0, max(df['RPS']) * 1.25)

# 막대 안쪽에 RPS 값 표시 (하얀색으로)
for i, (bar, rps) in enumerate(zip(bars, df['RPS'])):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height / 2,
             f'{rps:.1f}',
             ha='center', va='center', fontsize=11, fontweight='bold', color='white')

# 응답 시간 (꺾은선)
ax2 = ax1.twinx()
line1 = ax2.plot(x, df['Avg Response (ms)'], marker='o', color='orange', 
                 label='Avg Response Time', linewidth=2.5, markersize=9)
line2 = ax2.plot(x, df['95%ile (ms)'], marker='s', color='red', 
                 label='95%ile Response Time', linewidth=2.5, markersize=9)
ax2.set_ylabel('Response Time (ms)', fontsize=13, color='red', fontweight='bold')
ax2.tick_params(axis='y', labelcolor='red', labelsize=11)
ax2.set_ylim(0, max(df['Avg Response (ms)']) * 1.5)

# 응답 시간 값 표시 (위치 조정)
for i, (avg, p95) in enumerate(zip(df['Avg Response (ms)'], df['95%ile (ms)'])):
    # 평균 응답 시간 - 오른쪽에 표시
    ax2.text(i + 0.15, avg, f'{avg}ms', ha='left', va='center', 
             fontsize=9, color='orange', fontweight='bold',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='orange', alpha=0.8))
    # 95%ile - 왼쪽에 표시
    ax2.text(i - 0.15, p95, f'{p95}ms', ha='right', va='center', 
             fontsize=9, color='red', fontweight='bold',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='red', alpha=0.8))

# 제목 및 범례
plt.title('TeamMoa Load Test Results - Performance Metrics (Login Excluded)', 
          fontsize=16, fontweight='bold', pad=20)

# 범례 통합
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, 
          loc='upper left', fontsize=11, framealpha=0.95, edgecolor='gray')

# 통계 정보 추가
stats_text = f"Total Requests: {df['Total Requests'].sum():,} | Total Failures: {df['Failures'].sum()} | Overall Error Rate: {df['Failures'].sum() / df['Total Requests'].sum() * 100:.2f}%"
fig.text(0.5, 0.02, stats_text, ha='center', fontsize=10, 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.tight_layout(rect=[0, 0.04, 1, 1])
plt.show()

# 저장
output_path = 'load_test_summary.png'
fig.savefig(output_path, dpi=300, bbox_inches='tight')
print(f"\n✅ Chart saved: {output_path}")
print("📥 Download the file and upload to: docs/images/load_test_summary.png")